In [1]:
from pytube import YouTube
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
from PIL import Image
import copy
import os

In [2]:
# Video to process
game_name = 'clg_ggs_w6d2_s2019'
highlights_yt = 'https://www.youtube.com/watch?v=YRAgGKrPVLc'
game_yt = 'https://www.youtube.com/watch?v=_FGBtmfXzj0'

In [5]:
# Add folders
folders = ['frames', 'source', 'source/' + game_name, 'frames/' + game_name]
for f in folders:
    if not os.path.exists(f):
        os.makedirs(f)

In [6]:
# highlights
yt = YouTube(highlights_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)
def progress_fn(self, chunk, *_):
    pbar.update(len(chunk))
yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="source_high_res_highlights") 


 96%|█████████▌| 129040384/134240824 [00:07<00:00, 15550373.83it/s]

'./source/clg_ggs_w6d2_s2019/source_high_res_highlights.mp4'


100%|██████████| 134240824/134240824 [00:24<00:00, 15550373.83it/s]

In [7]:
# video
yt = YouTube(game_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)

yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="src") 



  0%|          | 0/759152504 [00:00<?, ?it/s]

  0%|          | 8192/759152504 [00:00<2:47:54, 75349.81it/s]

  0%|          | 1343488/759152504 [00:00<1:57:37, 107374.36it/s]

  1%|          | 5857280/759152504 [00:00<1:21:55, 153235.57it/s]

  2%|▏         | 14368768/759152504 [00:00<56:44, 218738.55it/s] 

  3%|▎         | 20389888/759152504 [00:00<39:27, 311997.77it/s]

  3%|▎         | 24780800/759152504 [00:00<27:32, 444276.55it/s]

  4%|▍         | 30556160/759152504 [00:00<19:13, 631526.21it/s]

  5%|▍         | 35168256/759152504 [00:00<13:28, 895222.57it/s]

  6%|▌         | 41779200/759152504 [00:01<09:24, 1271507.10it/s]

  6%|▌         | 46669824/759152504 [00:01<06:37, 1793649.44it/s]

  7%|▋         | 53567488/759152504 [00:01<04:40, 2511927.84it/s]

  8%|▊         | 60973056/759152504 [00:01<03:17, 3537041.55it/s]

  9%|▉         | 68714496/759152504 [00:01<02:19, 4955847.57it/s]

 10%|▉         | 74743808/759152504 [00:01<01:40, 6803021.11it/s]

 11%|█         | 8142

 88%|████████▊ | 666484736/759152504 [00:14<00:01, 49898493.76it/s]

 89%|████████▊ | 672776192/759152504 [00:14<00:01, 49419192.76it/s]

 89%|████████▉ | 677961728/759152504 [00:14<00:01, 44232454.22it/s]

 90%|█████████ | 684130304/759152504 [00:14<00:01, 47024856.22it/s]

 91%|█████████ | 690561024/759152504 [00:14<00:01, 51145709.58it/s]

 92%|█████████▏| 695951360/759152504 [00:14<00:01, 43519683.56it/s]

 93%|█████████▎| 702554112/759152504 [00:15<00:01, 47261186.66it/s]

 94%|█████████▎| 710557696/759152504 [00:15<00:00, 53849853.17it/s]

 94%|█████████▍| 716521472/759152504 [00:15<00:00, 50379293.03it/s]

 95%|█████████▌| 722010112/759152504 [00:15<00:01, 31915316.96it/s]

 96%|█████████▌| 727834624/759152504 [00:15<00:00, 36916205.40it/s]

 97%|█████████▋| 732635136/759152504 [00:15<00:00, 31501635.32it/s]

 97%|█████████▋| 736698368/759152504 [00:15<00:00, 33491379.05it/s]

 98%|█████████▊| 741048320/759152504 [00:16<00:00, 35968791.04it/s]

 98%|█████████▊| 746995712/7591525

'./source/clg_ggs_w6d2_s2019/src.mp4'



100%|██████████| 759152504/759152504 [00:32<00:00, 45258532.51it/s]

In [8]:
fps = stream.fps

In [9]:
# Crop image to get timestamp

def get_num_from_img(crop_img, blur_size=None):

    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    flip_img = 255 - crop_img

    # ret, thresh = cv2.threshold(flip_img, 127, 255, cv2.THRESH_BINARY) 
    ret, thresh = cv2.threshold(flip_img, 170, 255, cv2.THRESH_BINARY) 

    upscale_img = cv2.resize(thresh, (thresh.shape[1] * 20, thresh.shape[0] * 20), interpolation = cv2.INTER_AREA) 
    
    if not blur_size:
        blur_size = 25
        
    blur = cv2.blur(upscale_img,(blur_size,blur_size))
        
    
    pil_img = Image.fromarray(blur)
    text = pytesseract.image_to_string(pil_img,  config='outputbase digits')
        
    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    
    sharpened = cv2.filter2D(blur, -1, kernel_sharpening)
    pil_img = Image.fromarray(sharpened)
    if text == "":
        text = pytesseract.image_to_string(pil_img,  config='outputbase digits')

    
    pil_img.save("test.jpg", "JPEG")
    
#     if (text == "" and blur_size == None):
# #     0.8571428571428571
#         return (get_num_from_img(crop_img. blur_size + 10))
    
#     if text != "":
#         plt.figure()
#         plt.subplot(161),plt.imshow(crop_img),plt.title('Original')
#         plt.subplot(162),plt.imshow(flip_img),plt.title('Flipped')
#         plt.subplot(163),plt.imshow(thresh),plt.title('Threshold')
#         plt.subplot(164),plt.imshow(upscale_img),plt.title('Upscaled')
#         plt.subplot(165),plt.imshow(blur),plt.title('Blur ' + text)
#         plt.subplot(166),plt.imshow(sharpened),plt.title('Sharpened ' + text)
    return text

def get_timestamp_from_img(img):
    height, width, chan = img.shape

    crop_img = img[int(height * 0.068): int(height * 0.1), int(width * 0.485) : int(width * 0.52)]
    crop_img_left = img[int(height * 0.07): int(height * 0.095), int(width * 0.49) : int(width * 0.503)]
    crop_img_left = np.pad(crop_img_left, [(5, 5), (5, 5), (0,0)], mode='constant')
    crop_img_right = img[int(height * 0.07): int(height * 0.09), int(width * 0.504) : int(width * 0.515)]
    crop_img_right = np.pad(crop_img_right, [(5, 5), (5, 5), (0,0)], mode='constant')
    
    left = get_num_from_img(crop_img_left)
    try:
        left = int(left)
    except:
        left = ""
    right = get_num_from_img(crop_img_right)
    try:
        right = int(right)
    except:
        right = ""
    
    return (left, right)

def time_to_sec(time):
    minute, sec = time[0], time[1]
    return minute * 60 + sec
        

In [10]:
# Get the raw times
times = []
vidcap = cv2.VideoCapture('source/' + game_name + '/source_high_res_highlights.mp4')
count = 0
success = True
while success:
    success,image = vidcap.read()
    
    if count % 40 == 0:
        (left, right) = get_timestamp_from_img(image)
        times.append((left, right))
        print((left, right))

    count += 1

print(times)


('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
(9, '')
(9, '')
(9, '')
('', '')
('', '')
('', '')
('', 24)
(5, 23)
(5, 24)
(5, 26)
(5, 27)
(5, '')
(5, 30)
('', 31)
(5, 33)
(5, 34)
(5, 35)
(5, 37)
(5, 38)
(5, 0)
(5, 41)
(5, 41)
(7, 15)
(7, 17)
(7, 18)
(7, 20)
(7, 21)
(7, 22)
(7, 24)
(7, 59)
(8, 0)
(8, 1)
(8, 3)
(8, 4)
(8, 6)
(8, 7)
(8, 8)
(8, 9)
(8, 10)
('', 12)
(8, 13)
(8, 14)
(8, 28)
('', 29)
(8, 30)
(8, 32)
(8, 33)
(8, 34)
('', 36)
('', 37)
('', 38)
('', 40)
(0, 3)
(10, 4)
(10, 6)
(10, 7)
('', 9)
(10, 10)
(10, 11)
(10, 13)
(10, 14)
(12, 7)
(12, 3)
(12, 10)
(12, 11)
(12, 12)
(12, 14)
(12, 15)
(12, 17)
(12, 18)
(12, 19)
(12, 21)
(12, 22)
(12, 23)
(12, 24)
(12, 25)
(12, 27)
(12, 28)
(12, 30)
(12, 31)
(12, 32)
(12, 34)
(12, 35)
(13, 29)
(13, 30)
(13, 32)
(13, 33)
(13, 35)
(13, 36)
(13, 37)
(13, 39)
(13, '')
(13, 42)
(13, 43)
(13, '')
(13, 46)
(13, 47)
(13, 49)
(13, 49)
(15, 48)
(15, 49)
(15, 51)
(15, 52)
(15, 53)
(15, 55)
(15, 56)
(15, 57)
(15, 58)
(15, 

In [11]:
# Filter the first empty times (from non highlight footage)
saw_non_empty = False
times_first_non_empty = []
for time in times:
    if time[0] != "" and time[1] != "":
        saw_non_empty = True
    if time[0] == "" and time[1] == "":
        continue
    if saw_non_empty:
        times_first_non_empty.append(time)

last_time = None
# Guess the empty times
times_non_empty_temp = []
for time in times_first_non_empty:
    
    left = time[0]
    right = time[1]
    
    if last_time == None:
        last_time = [time[0], time[1]]
    else:
        # Going down in minutes, skip
        if left != "" and left < last_time[0]:
            continue
        if left != "":
            last_time[0] = left
        # New time but need to incr the minute
        if right != "" and right < last_time[1] and right < 5:
            last_time[0] += 1
        if right != "":
            last_time[1] = right
            
    times_non_empty_temp.append(copy.deepcopy(last_time))
    
# Filter to make sure none are empty
times_non_empty = []
for time in times_non_empty_temp:
    if (time[0] != "" and time[1] != ""):
        times_non_empty.append(time)
   
print(times_non_empty)


[[5, 23], [5, 24], [5, 26], [5, 27], [5, 27], [5, 30], [5, 31], [5, 33], [5, 34], [5, 35], [5, 37], [5, 38], [6, 0], [7, 15], [7, 17], [7, 18], [7, 20], [7, 21], [7, 22], [7, 24], [7, 59], [9, 0], [9, 12], [9, 29], [9, 36], [9, 37], [9, 38], [9, 40], [11, 4], [11, 9], [12, 7], [13, 3], [13, 29], [13, 30], [13, 32], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 39], [13, 42], [13, 43], [13, 43], [13, 46], [13, 47], [13, 49], [13, 49], [15, 48], [15, 49], [15, 51], [15, 52], [15, 53], [15, 55], [15, 56], [15, 57], [15, 58], [15, 59], [17, 1], [17, 2], [17, 6], [17, 31], [18, 1], [18, 47], [18, 48], [18, 50], [18, 51], [18, 52], [18, 54], [18, 55], [18, 57], [18, 58], [18, 59], [20, 1], [22, 9], [22, 9], [22, 12], [22, 14], [22, 15], [22, 16], [22, 18], [22, 18], [22, 20], [22, 21], [22, 22], [22, 24], [22, 25], [22, 27], [22, 28], [22, 29], [22, 31], [22, 32], [22, 34], [22, 35], [22, 55], [22, 57], [22, 53], [24, 0], [24, 0], [24, 2], [24, 3], [24, 5], [24, 6], [24, 7], [24, 8]

In [12]:
# Remove extremes
non_extreme_times = []
for i in range(len(times_non_empty)):
    left_diff = 0
    if i-1 > 0:
        left_diff = abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i-1]))
    right_diff = 0
    if i+1 < len(times_non_empty):
        right_diff =  abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i+1]))
    if left_diff > 10 and right_diff > 10:
        continue
    
    non_extreme_times.append(times_non_empty[i])
print(non_extreme_times)

[[5, 23], [5, 24], [5, 26], [5, 27], [5, 27], [5, 30], [5, 31], [5, 33], [5, 34], [5, 35], [5, 37], [5, 38], [7, 15], [7, 17], [7, 18], [7, 20], [7, 21], [7, 22], [7, 24], [9, 29], [9, 36], [9, 37], [9, 38], [9, 40], [11, 4], [11, 9], [13, 29], [13, 30], [13, 32], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 39], [13, 42], [13, 43], [13, 43], [13, 46], [13, 47], [13, 49], [13, 49], [15, 48], [15, 49], [15, 51], [15, 52], [15, 53], [15, 55], [15, 56], [15, 57], [15, 58], [15, 59], [17, 1], [17, 2], [17, 6], [18, 47], [18, 48], [18, 50], [18, 51], [18, 52], [18, 54], [18, 55], [18, 57], [18, 58], [18, 59], [22, 9], [22, 9], [22, 12], [22, 14], [22, 15], [22, 16], [22, 18], [22, 18], [22, 20], [22, 21], [22, 22], [22, 24], [22, 25], [22, 27], [22, 28], [22, 29], [22, 31], [22, 32], [22, 34], [22, 35], [22, 55], [22, 57], [22, 53], [24, 0], [24, 0], [24, 2], [24, 3], [24, 5], [24, 6], [24, 7], [24, 8], [24, 9], [24, 11], [24, 12], [24, 13], [24, 15], [24, 16], [24, 17], [24, 19],

In [13]:
timestamps = []

start_time = None
last_time = None
for i in range(len(non_extreme_times)):
    
    time = non_extreme_times[i]
    
    # Start of the highlight
    if start_time == None:
        start_time = time
    
    # Check if this is end of highlight
    if last_time != None and time_to_sec(time) - time_to_sec(last_time) > 10 or i == len(non_extreme_times)-1:
        timestamps.append((copy.copy(start_time), copy.copy(last_time)))
        start_time = copy.copy(time)
    
    last_time = time

print(timestamps)

[([5, 23], [5, 38]), ([7, 15], [7, 24]), ([9, 29], [9, 40]), ([11, 4], [11, 9]), ([13, 29], [13, 49]), ([15, 48], [15, 59]), ([17, 1], [17, 6]), ([18, 47], [18, 59]), ([22, 9], [22, 35]), ([22, 55], [22, 53]), ([24, 0], [24, 19]), ([26, 40], [26, 59]), ([29, 32], [29, 40]), ([30, 0], [30, 8]), ([30, 50], [31, 1])]


In [14]:
# Remove small timestamps and convert to sec
filtered_timestamps = []

for timestamp in timestamps:
    if (time_to_sec(timestamp[1]) - time_to_sec(timestamp[0])) > 5:
        filtered_timestamps.append(range(time_to_sec(timestamp[0]), time_to_sec(timestamp[1])))
print(filtered_timestamps)

[range(323, 338), range(435, 444), range(569, 580), range(809, 829), range(948, 959), range(1127, 1139), range(1329, 1355), range(1440, 1459), range(1600, 1619), range(1772, 1780), range(1800, 1808), range(1850, 1861)]


In [15]:
game_start_in_video_time = time_to_sec([7, 50]) - 8
print(game_start_in_video_time)

vidcap = cv2.VideoCapture('source/' + game_name + '/src.mp4')

print(vidcap.get(cv2.CAP_PROP_FPS))
fps = round(vidcap.get(cv2.CAP_PROP_FPS))
print('fps', fps)
count = 0
success = True
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_time = total_frames / fps
while success:
    success,image = vidcap.read()    
    
    if count % fps == 0:
        game_time = (count / fps) - game_start_in_video_time

        
        is_highlight = False
        print(str(game_time) + " / " str(total_time))
        for r in filtered_timestamps:
            if game_time in r:
                is_highlight = True
                break
            
        if is_highlight:
            cv2.imwrite('frames/' + game_name + '/frame%d-h.jpg' % (count / fps), image)

        else:
            cv2.imwrite('frames/' + game_name + '/frame%d-n.jpg' % (count / fps), image)

    count += 1


462
29.97
fps 30
-462.0
-461.0
-460.0
-459.0
-458.0
-457.0
-456.0
-455.0
-454.0
-453.0
-452.0
-451.0
-450.0
-449.0
-448.0
-447.0
-446.0
-445.0
-444.0
-443.0
-442.0
-441.0
-440.0
-439.0
-438.0
-437.0
-436.0
-435.0
-434.0
-433.0
-432.0
-431.0
-430.0
-429.0
-428.0
-427.0
-426.0
-425.0
-424.0
-423.0
-422.0
-421.0
-420.0
-419.0
-418.0
-417.0
-416.0
-415.0
-414.0
-413.0
-412.0
-411.0
-410.0
-409.0
-408.0
-407.0
-406.0
-405.0
-404.0
-403.0
-402.0
-401.0
-400.0
-399.0
-398.0
-397.0
-396.0
-395.0
-394.0
-393.0
-392.0
-391.0
-390.0
-389.0
-388.0
-387.0
-386.0
-385.0
-384.0
-383.0
-382.0
-381.0
-380.0
-379.0
-378.0
-377.0
-376.0
-375.0
-374.0
-373.0
-372.0
-371.0
-370.0
-369.0
-368.0
-367.0
-366.0
-365.0
-364.0
-363.0
-362.0
-361.0
-360.0
-359.0
-358.0
-357.0
-356.0
-355.0
-354.0
-353.0
-352.0
-351.0
-350.0
-349.0
-348.0
-347.0
-346.0
-345.0
-344.0
-343.0
-342.0
-341.0
-340.0
-339.0
-338.0
-337.0
-336.0
-335.0
-334.0
-333.0
-332.0
-331.0
-330.0
-329.0
-328.0
-327.0
-326.0
-325.0
-324.0
-323.0
-32

860.0
861.0
862.0
863.0
864.0
865.0
866.0
867.0
868.0
869.0
870.0
871.0
872.0
873.0
874.0
875.0
876.0
877.0
878.0
879.0
880.0
881.0
882.0
883.0
884.0
885.0
886.0
887.0
888.0
889.0
890.0
891.0
892.0
893.0
894.0
895.0
896.0
897.0
898.0
899.0
900.0
901.0
902.0
903.0
904.0
905.0
906.0
907.0
908.0
909.0
910.0
911.0
912.0
913.0
914.0
915.0
916.0
917.0
918.0
919.0
920.0
921.0
922.0
923.0
924.0
925.0
926.0
927.0
928.0
929.0
930.0
931.0
932.0
933.0
934.0
935.0
936.0
937.0
938.0
939.0
940.0
941.0
942.0
943.0
944.0
945.0
946.0
947.0
948.0
949.0
950.0
951.0
952.0
953.0
954.0
955.0
956.0
957.0
958.0
959.0
960.0
961.0
962.0
963.0
964.0
965.0
966.0
967.0
968.0
969.0
970.0
971.0
972.0
973.0
974.0
975.0
976.0
977.0
978.0
979.0
980.0
981.0
982.0
983.0
984.0
985.0
986.0
987.0
988.0
989.0
990.0
991.0
992.0
993.0
994.0
995.0
996.0
997.0
998.0
999.0
1000.0
1001.0
1002.0
1003.0
1004.0
1005.0
1006.0
1007.0
1008.0
1009.0
1010.0
1011.0
1012.0
1013.0
1014.0
1015.0
1016.0
1017.0
1018.0
1019.0
1020.0
1021.0
1022.0

2051.0
2052.0
2053.0
2054.0
2055.0
2056.0
2057.0
2058.0
2059.0
2060.0
2061.0
2062.0
2063.0
2064.0
2065.0
2066.0
2067.0
2068.0
2069.0
2070.0
2071.0
2072.0
2073.0
2074.0
2075.0
2076.0
2077.0
2078.0
2079.0
2080.0
2081.0
2082.0
2083.0
2084.0
2085.0
2086.0
2087.0
2088.0
2089.0
2090.0
2091.0
2092.0
2093.0
2094.0
2095.0
2096.0
2097.0
2098.0
2099.0
2100.0
2101.0
2102.0
2103.0
2104.0
2105.0
2106.0
2107.0
2108.0
2109.0
2110.0
2111.0
2112.0
2113.0
2114.0
2115.0
2116.0
2117.0
2118.0
2119.0
2120.0
2121.0
2122.0
2123.0
2124.0
2125.0
2126.0
2127.0
2128.0
2129.0
2130.0
2131.0
2132.0
2133.0
2134.0
2135.0
2136.0
2137.0
2138.0
2139.0
2140.0
2141.0
2142.0
2143.0
2144.0
2145.0
2146.0
2147.0
2148.0
2149.0
2150.0
2151.0
2152.0
2153.0
2154.0
2155.0
2156.0
2157.0
2158.0
2159.0
2160.0
2161.0
2162.0
2163.0
2164.0
2165.0
2166.0
2167.0
2168.0
2169.0
2170.0
2171.0
2172.0
2173.0
2174.0
2175.0
2176.0
2177.0
2178.0
2179.0
2180.0
2181.0
2182.0
2183.0
2184.0
2185.0
2186.0
2187.0
2188.0
2189.0
2190.0
2191.0
2192.0
2193.0

In [16]:
game_start_in_video_time = time_to_sec([14, 20])
print(game_start_in_video_time) - 10

# Add folders
src = 'source/' + game_name + '/src.mp4'
src_highlights = 'source/' + game_name + '/source_high_res_highlights.mp4'
os.remove(src)
os.remove(src_highlights)

860


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'